In [1]:
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import Spider
from scrapy.selector import Selector
from scrapy.loader import ItemLoader
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess

class Noticia(Item):
    titular = Field()
    descripcion = Field()
    id = Field()


class ElUniversoSpider(Spider):
    name = "MiSegundospider"

    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }  

    start_urls = ['https://www.eluniverso.com/deportes']

    # def parse(self, response):
    #     sel = Selector(response)
    #     noticias = sel.xpath('//div[contains(@class, "content-feed")]/ul/li')
    #     for i, elem in enumerate(noticias):
    #         item = ItemLoader(Noticia(), elem) 
    #         item.add_xpath('titular', './/h2/a/text()')
    #         item.add_xpath('descripcion', './/p/text()')
    #         item.add_value('id', i)
    #         yield item.load_item()

    def parse(self, response):
        soup = BeautifulSoup(response.body)
        contenedor_noticias=soup.find_all(class_="feed | divide-y relative")
        id = 0
        for contenedor in contenedor_noticias:
            # recursivo = false hijos directos, true recursivo en todos los elementos
            noticias = contenedor.find_all(class_='relative', recursive = False)
            for noticia in noticias:
                item = ItemLoader(Noticia(), response.body)
                titular = noticia.find('h2').text.replace('\n', '').replace('\r', '')
                descripcion = noticia.find('p')
                if (descripcion):
                    item.add_value('descripcion', descripcion.text.replace('\n', '').replace('\r', ''))
                else:
                    item.add_value('descripcion', 'N/A')
                item.add_value('titular', titular)
                item.add_value('id', id)
                id += 1
                yield item.load_item()

process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'datos_de_salida_3.json'
})

process.crawl(ElUniversoSpider)
process.start()


2024-08-21 14:50:36 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-08-21 14:50:36 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.11.7 (main, Dec 15 2023, 12:09:04) [Clang 14.0.6 ], pyOpenSSL 24.0.0 (OpenSSL 3.0.14 4 Jun 2024), cryptography 42.0.2, Platform macOS-10.16-x86_64-i386-64bit
2024-08-21 14:50:36 [scrapy.addons] INFO: Enabled addons:
[]
2024-08-21 14:50:36 [py.warnings] WARNING: /opt/anaconda3/lib/python3.11/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTE